In [ ]:
import pymongo, itertools, collections, unittest, time, sys
import networkx as nx
import matplotlib.pyplot as plt
from circos import CircosPlot
from hiveplot import HivePlot
%matplotlib inline
assertions = unittest.TestCase('__init__')

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017')
players, matches = client['usta'].players, client['usta'].matches
assert(players.count() == 76114)
assert(matches.count() == 6228)

In [ ]:
def nr_sets_completed(score):
    return sum([
            bool(int(s[0]) >= 6 or int(s[1]) >= 6)
            for s in score
        ])

def pretty_score(score):
    return '/'.join(['-'.join(s) for s in score])

def is_bagel(score):
    return all([not int(s[1]) for s in score])

def pretty_ratings(ratings):
    return ['{0:.3f}'.format(r) for r in ratings]

score = ['76', '64']
assert(nr_sets_completed(score) == 2)
assert(nr_sets_completed(['51']) == 0)
assert(nr_sets_completed(['16', '75']) == 2)
assert(pretty_score(score) == '7-6/6-4')
assert(is_bagel(score) == False)
assert(is_bagel(['60', '30']) == True)

In [ ]:
def add_player(player, graph):
    player_id = int(player['_id'])
    full_name = ' '.join([player['first_name'], player['last_name']])
    rating = player['rating_level'] - 0.25 # start in the middle
    kwargs = dict(
        name=full_name, rating=rating,
        level='{0:.1f}'.format(player['rating_level'])            
    )
    graph.add_node(player_id, **kwargs)

In [ ]:
def get_player_graph(player, graph, verbose=False):
    player_id = int(player['_id'])
    global nesting
    indent = ' '*nesting
    player_full_name = ' '.join([player['first_name'], player['last_name']])
    if verbose:
        print('{}{}: {} ({})'.format(
            indent, player_id, player_full_name, player['rating_level']
        ))
    query = {'$or': []}
    # sd_keys, wl_keys = ['singles', 'doubles'], ['winner', 'loser']
    sd_keys, wl_keys = ['singles'], ['winner', 'loser']
    for key in itertools.product(sd_keys, wl_keys):
        query['$or'].append({'.'.join(key): player_id})
    player_matches = matches.find(query).sort('date')
    if player_matches.count() < 2:
        if verbose:
            print('{}skip {}. Only played 1 match'.format(indent, player_full_name))
        return
    for match in player_matches:
        individual_match_found = False
        for sd in sd_keys:
            if individual_match_found:
                break
            for individual_match in match[sd]:
                if individual_match_found:
                    break
                for iwl, wl in enumerate(wl_keys):
                    if individual_match_found:
                        break
                    if individual_match[wl] == player_id or (
                        isinstance(individual_match[wl], list) and \
                        player_id in individual_match[wl]
                    ):
                        individual_match_found = True
                        score = individual_match['score']
                        opponent_id = individual_match[wl_keys[int(not(iwl))]]
                        win_or_loss = wl[0].upper()
                        if (
                            nr_sets_completed(score) < 2 or # two sets completed
                            is_bagel(score) or opponent_id is None # skip bagels and defaults
                        ):
                            break
                        opponent = players.find_one({'_id': str(opponent_id)})
                        if opponent_id not in graph.node:
                            add_player(opponent, graph)
                        kwargs = dict(score=score, date=match['date'])
                        source, sink = (player_id, opponent_id) if win_or_loss == 'W' \
                                else (opponent_id, player_id)
                        if sink in graph.edge[source] and kwargs in graph[source][sink].values():
                            break # match already added
                        graph.add_edge(source, sink, **kwargs)
                        if verbose:
                            print(
                                '{}{}:'.format(indent, match['_id']), win_or_loss,
                                pretty_score(score), 'vs',
                                ' '.join([opponent['first_name'], opponent['last_name']]),
                                '({})'.format(opponent_id)
                            )
                        nesting += 1
                        if not verbose:
                            sys.stdout.write('\r{}'.format(nesting))
                        get_player_graph(opponent, graph=graph, verbose=verbose)
                        nesting -= 1

In [ ]:
sys.setrecursionlimit(3000)
nesting = 0
G = nx.MultiDiGraph()
player = players.find_one({'last_name': 'Huck', 'first_name': 'Patrick'})
add_player(player, G) # root node
get_player_graph(player, graph=G, verbose=False)

In [ ]:
print(len(G))

In [ ]:
def crd(score):
    """
    CRD - Computer Rated Differential (numerical value for the difference in score)
    
    One way to assign a value to a specific score, is to count the number of service breaks
    and scale it to a value appropriate for NTRP ratings. For instance, at-level/true 4.5
    players should populate the core of the 4.5 interval. Defining the core of a 0.5-wide
    interval as its inner 90%, yields the range 4.05 - 4.45. An average upper 4.5 player
    would then correspond to a 4.35 rating and a lower 4.5 player to 4.15.
    
    A good scale for the CRD reflects the fact that an upper 4.5 player routinely beats a
    lower 4.5 player. A sensible choice for a routine but competitive win is a score of
    6-3/6-3 or 6-3/6-2 [see below].
    
    The number of service breaks in these cases should hence be equivalent to the difference
    of 0.2 between an upper and a lower 4.5 player. A 6-3/6-3 win entails 3 service breaks
    whereas a 6-3/6-2 win could be counted as 3.5 service breaks. Assigning a scaling factor
    of 0.06 for each additional service break is thus a good choice and results in CRDs of
    0.18 and 0.21, respectively.
    
    In the CRD, the number of service breaks in the third set also counts half since the
    opponents are basically even but the loser of the third set doesn't get a chance to even
    out in a fourth.
    
    A match is considered competitive if the loser plays one competitive set (>= 3 games) or
    scores at least 4 games in total. If this isn't the case, the outcome of the match would
    almost always be a non-competitive score regardless of how often they play each other.
    Since neither player probably plays their best due to the non-competitiveness of the match,
    the score likely is inaccurate.
    
    In the worst case of a 6-0/6-0 score, it's tough to assess performance of either player
    at all without a game on the scoreboard for the loser. The loser might have been close
    to scoring a game in every game of the match or in none at all.
    
    It hence makes sense to treat 6-0/6-0 scores as outliers in the calculation of ratings
    and discard them. But we can use this score to estimate the score inaccuracy of a
    non-competitive match. In such a match, the winner scored 6 service breaks, and an
    additional game for the loser would correspond to half a service break difference in
    the CRD, i.e. 0.03. Say the loser got close to winning one of the winner's 6 service
    games. The change in CRD value equivalent to this performance would be 0.03/6 = 0.005.
    For non-competitive matches, the CRD is hence adjusted downward by 0.005 to not affect
    both the players ratings as much as two competitive sets [which would have been more fun
    for both]. 
    
    See http://web.archive.org/web/20051211104109/http://www.wetennis.com/rate.htm
    """
    if score == ['60', '60']:
        raise ValueError('{} should be ignored!'.format(pretty_score(score)))
    crd = 0 # computer rated differential (CRD)
    scf = 0.06 # scaling factor = CRD equivalent for one service break
    competitive_set = False # one competitive set played (> 2 games)
    glt = 0 # number of games scored by loser
    for i,s in enumerate(score):
        gw, gl = map(int, s) # games winner and loser
        nb = float(gw-gl)/2 # number of breaks
        crd += (nb/2 if i == 2 else nb) * scf # third set counts half
        glt += gl
        if i < 2 and gl > 2: # skip third set
            competitive_set = True
    return crd if competitive_set or glt > 3 else crd-scf/12

# competitive 3-set matches
assertions.assertAlmostEqual(crd(['75', '57', '10']), 0.015)
assertions.assertAlmostEqual(crd(['64', '46', '64']), 0.03)
assertions.assertAlmostEqual(crd(['75', '57', '75']), 0.03)
assertions.assertAlmostEqual(crd(['75', '57', '63']), 0.045)
assertions.assertAlmostEqual(crd(['75', '57', '61']), 0.075)
# competitive 2-set matches
# one competitive set (>= 3 games) or
# total >= 4 games
assertions.assertAlmostEqual(crd(['76', '76']), 0.06)
assertions.assertAlmostEqual(crd(['76', '75']), 0.09)
assertions.assertAlmostEqual(crd(['76', '64']), 0.09)
assertions.assertAlmostEqual(crd(['75', '64']), 0.12)
assertions.assertAlmostEqual(crd(['64', '64']), 0.12)
assertions.assertAlmostEqual(crd(['64', '63']), 0.15)
assertions.assertAlmostEqual(crd(['63', '63']), 0.18)
assertions.assertAlmostEqual(crd(['75', '62']), 0.18)
assertions.assertAlmostEqual(crd(['62', '63']), 0.21)
assertions.assertAlmostEqual(crd(['62', '62']), 0.24)
assertions.assertAlmostEqual(crd(['63', '61']), 0.24)
assertions.assertAlmostEqual(crd(['61', '63']), 0.24)
assertions.assertAlmostEqual(crd(['63', '60']), 0.27)
assertions.assertAlmostEqual(crd(['60', '63']), 0.27)
# non-competitive matches
assertions.assertAlmostEqual(crd(['62', '61']), 0.265)
assertions.assertAlmostEqual(crd(['62', '60']), 0.295)
assertions.assertAlmostEqual(crd(['61', '61']), 0.295)
assertions.assertAlmostEqual(crd(['61', '60']), 0.325)

# crd(['60', '60'])

In [ ]:
edges = sorted(G.edges(data=True), key=lambda x: x[-1]['date'])
drv = {} # dynamic rating values
for winner_id, loser_id, d in edges:
    if winner_id not in drv:
        drv[winner_id] = [G.node[winner_id]['rating']]
    if loser_id not in drv:
        drv[loser_id] = [G.node[loser_id]['rating']]
    prd = drv[winner_id][-1] - drv[loser_id][-1] # Player Rating Differential
    rdd = crd(d['score']) - prd # Rating Differential Discrepancy
    awr = drv[winner_id][-1] + rdd/2 # Adjusted Winner’s Rating (before averaging)
    dwr = (sum(drv[winner_id]) + awr) / (len(drv[winner_id]) + 1) # Dynamic Winner’s Rating
    drv[winner_id].append(dwr)
    alr = drv[loser_id][-1] - rdd/2 # Adjusted Loser’s Rating (before averaging)
    dlr = (sum(drv[loser_id]) + alr) / (len(drv[loser_id]) + 1) # Dynamic Loser’s Rating
    drv[loser_id].append(dlr)

In [ ]:
fig = plt.figure(figsize=(17,8))
ax = fig.add_subplot(111)
for k,v in drv.items():
    if len(v) > 4:
        plt.plot(v)
plt.show()

In [ ]:
# nx.draw(G)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)

nodes = sorted(G.nodes())
edges = G.edges()
node_cmap = {'4.0': 'blue', '4.5': 'red', '5.0': 'green'}
nodecolors = [node_cmap[G.node[n]['level']] for n in G.nodes()]

c = CircosPlot(nodes, edges, radius=10, ax=ax, fig=fig, nodecolor=nodecolors)
c.draw()

In [ ]:
nodes = dict(
    (level, [n for n,d in G.nodes(data=True) if d['level'] == level])
    for level in ['4.0', '4.5', '5.0']
)
edges = dict(group1=G.edges(data=True))
edge_cmap = dict(group1='black')
h = HivePlot(nodes, edges, node_cmap, edge_cmap)
h.draw()

In [ ]:
# sorted([(n, G.neighbors(n)) for n in G.nodes()], key=lambda x: len(x[1]), reverse=True)
# print(nx.degree_centrality(G))
# print(nx.has_path(G, 400, 1))
fig = plt.figure(0)
degree_centralities = list(nx.degree_centrality(G).values())
plt.hist(degree_centralities)
plt.title('Degree Centralities')